# SQL Murder Mystery

The sql_murder_mystery database
A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQL City​. Start by retrieving the corresponding crime scene report from the police department’s database.

<img src="https://mystery.knightlab.com/174092-clue-illustration.png?raw=true" width=100%/>

All the clues to this mystery are buried in a huge database, and you need to use SQL to navigate through this vast network of information. Your first step to solving the mystery is to retrieve the corresponding crime scene report from the police department’s database. Below we'll explain from a high level the commands you need to know; whenever you are ready, you can start adapting the examples to create your own SQL commands in search of clues -- you can run any SQL in any of the code boxes, no matter what was in the box when you started.

Image
Below is an Entity Relationship Diagram (ERD) of the sql-murder-mystery database:

<img src="https://mystery.knightlab.com/schema.png?raw=true" width=100%/>

Image
As can be seen from the ERD, the sql_murder_mystery database consists of 9 tables containing information about movies, cast, genre, and so much more.

Let's get started!

## Loading the database

Before you begin, you need to prepare your SQL environment.  You can do this by loading the magic command `%load_ext sql`.

In [1]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook. 
# If you get an error here, make sure that mysql and pymysql are installed correctly. 

%load_ext sql

Next, go ahead and load your database. To do this, you will need to ensure you have downloaded the `sql-murder-mystery.db` sqlite file from Athena and have stored it in a known location.

In [2]:
# Establish a connection to the local database using the '%sql' magic command.
# Replace 'password' with our connection password and `db_name` with our database name. 
# If you get an error here, please make sure the database name or password is correct.

%sql sqlite:///../databases/sql-murder-mystery.db

'Connected: @../databases/sql-murder-mystery.db'

### View all tables in the database

In [3]:
# View all tables present in the database
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///../databases/sql-murder-mystery.db
Done.


name
crime_scene_report
drivers_license
facebook_event_checkin
interview
get_fit_now_member
get_fit_now_check_in
solution
income
person


## The investigation begins.

Examine the crime scene report for details such as the date (January 15, 2018) and location (SQL City) to establish a foundation for the investigation.

The report reveals that there were two witnesses to the incident. 

In [4]:
%%sql
/* Step retrieve the corresponding crime scene report*/

SELECT *
FROM crime_scene_report
WHERE date = 20180115
AND type = 'murder'
AND city LIKE "SQL%";

 * sqlite:///../databases/sql-murder-mystery.db
Done.


date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


On review of the witness statements and crime scene report there is a connection to a local gym, which is a potential lead in the investigation.

In [5]:
%%sql
/* Check interview table for witness statements*/

SELECT p.name, i.transcript
FROM person p
JOIN interview i
ON p.id = i.person_id
WHERE (p.name LIKE "Annabel %" 
AND p.address_street_name LIKE "%Franklin%")
OR p.name = (
    SELECT p.name
    FROM person p
    JOIN interview as i
    ON p.id = i.person_id
    WHERE p.address_street_name LIKE "Northwestern Dr%"
    ORDER BY p.address_number DESC
    LIMIT 1
);

 * sqlite:///../databases/sql-murder-mystery.db
Done.


name,transcript
Morty Schapiro,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
Annabel Miller,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


Based on the witness testimony, query the gym records to find a gym member with a gold membership status, membership number starting with "48Z", who drives a car with a plate number that included "H42W".  

In [6]:
%%sql
/*To find the potential killer, join and query the drivers_license, get_fit_now_member, get_fit_now_check_in and person tables*/

SELECT m.name, m.id, m.membership_status
FROM get_fit_now_member m
JOIN get_fit_now_check_in ci
ON m.id = ci.membership_id
JOIN person p
ON p.id = m.person_id
JOIN drivers_license dl
ON dl.id = p.license_id
WHERE m.id LIKE "48Z%"
AND m.membership_status = "gold"
AND dl.plate_number LIKE "%H42W%";

 * sqlite:///../databases/sql-murder-mystery.db
Done.


name,id,membership_status
Jeremy Bowers,48Z55,gold


Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries.

To find the real villian, review the transcript of the interview with Jeremy Bowers. 

In [7]:
%%sql

SELECT p.name, i.transcript
FROM person p
JOIN interview i
ON p.id = i.person_id
WHERE p.name = "Jeremy Bowers";

 * sqlite:///../databases/sql-murder-mystery.db
Done.


name,transcript
Jeremy Bowers,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


Jeremy was hired by a lady with red hair who owns a Tesla Model S. To identify the lady, join and query the events, person and drivers license tables.

In [8]:
%%sql

SELECT DISTINCT(p.name)
FROM facebook_event_checkin fb
JOIN person p
ON fb.person_id = p.id
JOIN drivers_license dl
ON p.license_id = dl.id
WHERE fb.date like '201712%'
AND dl.hair_color = 'red'
AND dl.car_make = 'Tesla'
AND dl.car_model = 'Model S';

 * sqlite:///../databases/sql-murder-mystery.db
Done.


name
Miranda Priestly


Miranda Priestly is the Villian behind the scenes. She hired Jeremy Bowers to commit the crime.

## THE END.